Constant Velocity: Compute the mean velocity (vxi, vyi) from t = {1, ..., Tobs} and then forecast (xt_i, yt_i) for t = {Tobs+1, ..., Tpred} using (vxi, vyi) as the constant velocity.

first, load in data

In [1]:
import numpy as np
train_file = np.load('data/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)
test_file = np.load('data/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


visualization code

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# make gif out of a scene.
def make_gif(data_matrix, name='example'):
   cmap = plt.cm.get_cmap('viridis', 50)
   fig, ax = plt.subplots(figsize=(10, 10))
   # Function to update plot for each frame
   def update(frame):
       ax.clear()

       # Get data for current timestep
       for i in range(1, data_matrix.shape[0]):
           x = data_matrix[i, frame, 0]
           y = data_matrix[i, frame, 1]
           if x != 0 and y != 0:
               xs = data_matrix[i, :frame+1, 0]  # Include current frame
               ys = data_matrix[i, :frame+1, 1]  # Include current frame
               # trim all zeros
               mask = (xs != 0) & (ys != 0)  # Only keep points where both x and y are non-zero
               xs = xs[mask]
               ys = ys[mask]

               # Only plot if we have points to plot
               if len(xs) > 0 and len(ys) > 0:
                   color = cmap(i)
                   ax.plot(xs, ys, alpha=0.9, color=color)
                   ax.scatter(x, y, s=80, color=color)

       ax.plot(data_matrix[0, :frame, 0], data_matrix[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
       ax.scatter(data_matrix[0, frame, 0], data_matrix[0, frame, 1], s=80, color='tab:orange')
       # Set title with timestep
       ax.set_title(f'Timestep {frame}')
       # Set consistent axis limits
       ax.set_xlim(data_matrix[:,:,0][data_matrix[:,:,0] != 0].min() - 10, data_matrix[:,:,0][data_matrix[:,:,0] != 0].max() + 10)
       ax.set_ylim(data_matrix[:,:,1][data_matrix[:,:,1] != 0].min() - 10, data_matrix[:,:,1][data_matrix[:,:,1] != 0].max() + 10)
       ax.legend()

       return ax.collections + ax.lines

   # Create animation
   anim = animation.FuncAnimation(fig, update, frames=list(range(0, data_matrix.shape[1], 3)),
                               interval=100, blit=True)
   # Save as GIF
   anim.save(f'trajectory_visualization_{name}.gif', writer='pillow')
   plt.close()
data_matrix = train_data[0]        
make_gif(data_matrix, 'index0')

/var/folders/wv/j6ybhf8906j4bmnp171wtn680000gn/T/ipykernel_96938/2021253749.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('viridis', 50)


Preprocess data

handle this: Many trajectories are not complete, and have been padded with 0s. this can also happen when there are less than 50 actors

In [6]:
def preprocess_data(data):
    """
    Removes padded agents (agents with all zero values across time steps).
    
    Args:
        data (numpy.ndarray): Shape (scenarios, agents, time_steps, dimensions)
    
    Returns:
        numpy.ndarray: Filtered dataset without padded agents.
    """
    scenarios, agents, time_steps, dimensions = data.shape
    processed_data = []

    for i in range(scenarios):
        scenario_data = data[i]  # Shape (agents, time_steps, dimensions)
        
        # Identify non-padded agents (at least one nonzero value across all time steps)
        valid_agents = np.any(scenario_data != 0, axis=(1, 2))  # Shape (agents,)
        
        # Filter out only the valid agents
        filtered_agents = scenario_data[valid_agents]  # Shape (valid_agents, time_steps, dimensions)
        
        processed_data.append(filtered_agents)

    return processed_data  # List of variable-length arrays per scenario

train_data_processed = preprocess_data(train_data)
test_data_processed = preprocess_data(test_data)

# Print results
print(f"Original Train Data Shape: {train_data.shape}")
print(f"Processed Train Data Length: {len(train_data_processed)} (variable agents per scenario)")

print(f"Original Test Data Shape: {test_data.shape}")
print(f"Processed Test Data Length: {len(test_data_processed)} (variable agents per scenario)")


Original Train Data Shape: (10000, 50, 110, 6)
Processed Train Data Length: 10000 (variable agents per scenario)
Original Test Data Shape: (2100, 50, 50, 6)
Processed Test Data Length: 2100 (variable agents per scenario)


may want to add handling incomplete trajectories (some kind of interpolation to fill in missing values)

In [ ]:
#todo: handle missing trajectories

Compute mean velocity for each agent. use this to forecase the position

In [13]:
import numpy as np

def forecast_positions(scenario_data, Tobs, Tpred):
    """
    Computes mean velocity for each agent and forecasts future positions.
    
    Args:
        scenario_data (numpy.ndarray): Shape (agents, time_steps, dimensions)
        Tobs (int): Number of observed time steps
        Tpred (int): Number of future time steps to predict
    
    Returns:
        numpy.ndarray: Predicted positions of shape (agents, Tpred, 2) -> (x, y)
    """
    agents, time_steps, dimensions = scenario_data.shape
    assert Tobs <= time_steps, "Tobs cannot exceed total time steps."

    # Extract observed positions at Tobs (last observed position)
    last_x = scenario_data[:, Tobs-1, 0]  # Last observed x position
    last_y = scenario_data[:, Tobs-1, 1]  # Last observed y position

    # Compute mean velocities from observed data
    velocity_x = scenario_data[:, :Tobs, 2]
    velocity_y = scenario_data[:, :Tobs, 3]

    vx_mean = np.mean(velocity_x, axis=1)  # Mean vx per agent
    vy_mean = np.mean(velocity_y, axis=1)  # Mean vy per agent

    # Time offsets for future predictions
    future_time_steps = np.arange(1, Tpred + 1)  # 1, 2, ..., Tpred

    # Compute future positions using constant velocity assumption
    future_x = last_x[:, np.newaxis] + future_time_steps * vx_mean[:, np.newaxis]
    future_y = last_y[:, np.newaxis] + future_time_steps * vy_mean[:, np.newaxis]

    # Stack predictions into shape (agents, Tpred, 2)
    predicted_positions = np.stack((future_x, future_y), axis=-1)

    return predicted_positions

Make predictions on train, and evaluate MSE

In [16]:
from sklearn.metrics import mean_squared_error

def evaluate_mse(train_data, Tobs=50, Tpred=60):
    """
    Computes the constant velocity prediction for the ego agent in train_data and evaluates MSE.
    
    Args:
        train_data (numpy.ndarray): Training data of shape (N, 50, 110, 6).
        Tobs (int, optional): Number of observed time steps. Defaults to 50.
        Tpred (int, optional): Number of future time steps to predict. Defaults to 60.

    Returns:
        float: Mean Squared Error of predicted vs actual future positions.
    """
    N = train_data.shape[0]  # Number of scenarios
    mse_list = []

    for i in range(N):
        scenario_data = train_data[i]  # Shape (50, 110, 6)
        
        # Extract the ego agent (agent 0)
        ego_agent_data = scenario_data[0]  # Shape (110, 6)
        
        # Get ground truth future positions (Tobs:Tobs+Tpred)
        ground_truth = ego_agent_data[Tobs:Tobs+Tpred, :2]  # Shape (60, 2) -> (x, y)

        # Forecast future positions for the ego agent
        predicted_positions = forecast_positions(ego_agent_data[np.newaxis, :, :], Tobs, Tpred)  # (1, 60, 2)

        # Compute MSE
        mse = mean_squared_error(ground_truth, predicted_positions[0])
        mse_list.append(mse)

    # Compute mean of all scenario MSEs
    overall_mse = np.mean(mse_list)
    
    print(f"Mean Squared Error (MSE) on train data: {overall_mse:.4f}")
    return overall_mse

# Evaluate MSE
evaluate_mse(train_data)

Mean Squared Error (MSE) on train data: 24704.1730


np.float64(24704.1729997388)

In [14]:
import pandas as pd

In [15]:
def generate_submission(data, output_csv, Tobs=50, Tpred=60):
    """
    Loads test data, applies constant velocity forecasting to the ego agent, and saves results as CSV.
    
    Args:
        test_data_path (str): Path to the .npz file containing test data.
        output_csv (str): Path to save the output CSV.
        Tobs (int, optional): Number of observed time steps. Defaults to 50.
        Tpred (int, optional): Number of future time steps to predict. Defaults to 60.
    """

    # Initialize array for predictions
    submission_data = np.zeros((data.shape[0], 1, Tpred, 2))  # (2100, 1, 60, 2)

    # Iterate over all test scenarios
    for i in range(data.shape[0]):
        scenario_data = data[i]  # Shape (50, 50, 6)
        
        # Extract the ego agent (agent 0)
        ego_agent_data = scenario_data[0]  # Shape (50, 6)
        
        # Forecast future positions for the ego agent
        predicted_positions = forecast_positions(ego_agent_data[np.newaxis, :, :], Tobs, Tpred)  # (1, 60, 2)
        
        # Store in final array
        submission_data[i, 0] = predicted_positions[0]

    # Convert to DataFrame and save as CSV
    submission_df = pd.DataFrame(submission_data.reshape(data.shape[0], -1))  # Flatten (2100, 120)
    submission_df.to_csv(output_csv, index=False, header=False)
    
    print(f"Submission file '{output_csv}' saved successfully!")

generate_submission(test_data, 'constant_velocity_submission.csv')

Submission file 'constant_velocity_submission.csv' saved successfully!
